<a href="https://colab.research.google.com/github/shrut9/BERT-NER-REUME-PARSER/blob/main/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# train.py

In [ ]:
pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=9ad396eedd39cb4e352c31f976fd1bbaa3192255ff0ad9e069bcd0f68ea8378b
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
pip install pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 29.8 MB/s eta 0:00:00


In [ ]:
pip install transformers

In [ ]:
pip install torch

In [ ]:
import argparse
import numpy as np
import torch
from transformers import BertForTokenClassification, BertTokenizerFast
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from sample_data.utils import trim_entity_spans, convert_goldparse, ResumeDataset, tag2idx, idx2tag, get_hyperparameters, train_and_val_model

# Instead of parsing arguments, directly assign values
# parser = argparse.ArgumentParser(description='Train Bert-NER')
# parser.add_argument('-e', type=int, default=5, help='number of epochs')
# parser.add_argument('-o', type=str, default='.',
#                     help='output path to save model state')
#
# args = parser.parse_args().__dict__

# Directly set values for EPOCHS and output_path
EPOCHS = 5  # or any desired value
output_path = "."  # or any desired output path

MAX_LEN = 500
MAX_GRAD_NORM = 1.0
MODEL_NAME = 'bert-base-uncased'
TOKENIZER = BertTokenizerFast('./vocab/vocab.txt', lowercase=True)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data = trim_entity_spans(convert_goldparse('sample_data/Resumes.json'))

total = len(data)
train_data, val_data = data[:180], data[180:]

# Assuming 'tokens' is at index 0 in each inner list
def get_sequence_length(example):
    return len(example[0]) # Accessing the 'tokens' list using index 0

# Sort data by sequence length
train_data.sort(key=get_sequence_length)
val_data.sort(key=get_sequence_length)

train_d = ResumeDataset(train_data, TOKENIZER, tag2idx, MAX_LEN)
val_d = ResumeDataset(val_data, TOKENIZER, tag2idx, MAX_LEN)

train_sampler = RandomSampler(train_d)
train_dl = DataLoader(train_d, sampler=train_sampler, batch_size=1)

val_dl = DataLoader(val_d, batch_size=4)

model = BertForTokenClassification.from_pretrained(
    MODEL_NAME, num_labels=len(tag2idx))
model.to(DEVICE)
optimizer_grouped_parameters = get_hyperparameters(model, True)
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

def train_and_val_model(model, tokenizer, optimizer, epochs, idx2tag, tag2idx, max_grad_norm, device, train_dataloader, valid_dataloader):
    # ... [other parts of the function remain the same]

    for b_input_ids, b_input_mask, b_labels in batch:
        # ... [other parts of the loop remain the same]

        # Reshape logits to (batch_size, sequence_length, num_labels)
        tr_logits = tr_logits.reshape(b_input_ids.shape[0], b_input_ids.shape[1], -1)

        # Create preds_mask with shape (batch_size, sequence_length)
        preds_mask = b_input_mask.cpu().detach().numpy()

        # Iterate over each example in the batch
        for i in range(b_input_ids.shape[0]):
            # Get logits and mask for the current example
            example_logits = tr_logits[i]
            example_mask = preds_mask[i]

            # Apply mask to logits and get predictions for valid tokens
            valid_logits = example_logits[example_mask == 1]  # Select logits where mask is True
            tr_batch_preds = np.argmax(valid_logits, axis=1)

            # ... [rest of the loop remains the same, using tr_batch_preds]

    # ... [rest of the function remains the same]
torch.save(
    {
        "model_state_dict": model.state_dict()
    },
    f'{output_path}/model-state.bin',
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# app.py

In [ ]:
pip install waitress

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 5.0 MB/s eta 0:00:00


In [ ]:
import io
import argparse
import torch
from transformers import BertTokenizerFast, BertForTokenClassification
from flask import Flask, jsonify, request
from server.utils import preprocess_data, predict, idx2tag
from waitress import serve

app = Flask(__name__)
app.config['JSON_SORT_KEYS'] = False

MAX_LEN = 500
NUM_LABELS = 12
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = 'bert-base-uncased'
STATE_DICT = torch.load("model-state.bin", map_location=DEVICE)
TOKENIZER = BertTokenizerFast("./vocab/vocab.txt", lowercase=True)

model = BertForTokenClassification.from_pretrained(
    'bert-base-uncased', state_dict=STATE_DICT['model_state_dict'], num_labels=NUM_LABELS)
model.to(DEVICE)


@app.route('/predict', methods=['POST'])
def predict_api():
    if request.method == 'POST':
        data = io.BytesIO(request.files.get('resume').read())
        resume_text = preprocess_data(data)
        entities = predict(model, TOKENIZER, idx2tag,
                           DEVICE, resume_text, MAX_LEN)
        return jsonify({'entities': entities})


if __name__ == '__main__':
    serve(app, host='0.0.0.0', port=5000)

<ipython-input-7-0cbcc6762441>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  STATE_DICT = torch.load("model-state.bin", map_location=DEVICE)
